Librerias

In [53]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException, NoSuchElementException
import time
from datetime import datetime, timedelta
import pandas as pd
import re
from selenium.webdriver.chrome.options import Options
import undetected_chromedriver as uc

Extraigo primero los enlaces de los torneos.

In [61]:
try:
    if driver is not None:
        driver.quit()
except:
    pass  # Por si driver aún no estaba creado

options = Options()
options.add_argument("--headless")

driver = webdriver.Chrome(options=options)
driver.get("https://www.flashscore.com/tennis")

try:
    wait = WebDriverWait(driver, 10)

    # Esperar manualmente a que se resuelva el CAPTCHA
    #input("Resuelve el CAPTCHA (y otras cosas) manualmente en el navegador y presiona Enter aquí para continuar...")

    time.sleep(2)


    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.leftMenu__item")))
    torneos = driver.find_elements(By.CSS_SELECTOR, "div.leftMenu__item")
    enlaces = []

    for torneo in torneos:
        #print("Texto",torneo.text)

        if not "Show more" in torneo.text:
            # Obtén el título del torneo (opcional)
            titulo = torneo.get_attribute("title")
            #print("Título:", titulo)

            # Localiza el enlace (<a>) dentro del elemento "leftMenu_item"
            enlace = torneo.find_element(By.CSS_SELECTOR, "a")
            href = enlace.get_attribute("href")
            if not "calendar" in href and not "rankings" in href and not "camref" in href and not "davis" in href and not "doubles" in href:
                enlaces.append({
                        "href": href,
                        "nombre": titulo
                    })
                #print("Enlace:", href)

except Exception as e:
    print("Error:", e)
    
finally:
    driver.quit()

Extracción

In [ ]:
try:
    if driver is not None:
        driver.quit()
except:
    pass  # Por si driver aún no estaba creado


options = uc.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-blink-features=AutomationControlled")

driver = uc.Chrome(options=options)

driver.get("https://www.flashscore.com/tennis")

try:
    wait = WebDriverWait(driver, 10)

    # Esperar manualmente a que se resuelva el CAPTCHA
    #input("Resuelve el CAPTCHA (y otras cosas) manualmente en el navegador y presiona Enter aquí para continuar...")

    time.sleep(2)

    # Variables
    base_url = "https://www.flashscore.com/tennis"

    datos = []

    jugadoresA = []
    jugadoresB = []
    setsA = []
    setsB = []
    fecha = []
    retirado = []

    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.leftMenu__item")))
    torneos = driver.find_elements(By.CSS_SELECTOR, "div.leftMenu__item")


    for j,e in enumerate(enlaces[len(enlaces) - 2:]):
        href = e['href']
        nombre = e['nombre']
        enlaces_partidos = []
        url = f"{href}results/"
        driver.get(url)
        print("Dentro de", url)

        try:
            wait.until(EC.presence_of_element_located((
                By.CSS_SELECTOR, ".event__match.event__match--withRowLink.event__match--static.event__match--twoLine"
            )))
            partidos = driver.find_elements(
                By.CSS_SELECTOR, ".event__match.event__match--withRowLink.event__match--static.event__match--twoLine"
            )
            print("Número de partidos:", len(partidos), "en el enlace", j, "de", len(enlaces), "enlaces")
        except TimeoutException:
            print("No se encontraron partidos en el enlace", j, "de", len(enlaces), "enlaces")
            partidos = []  # Por si luego recorres esta lista

        for partido in partidos:
            try:
                enlace = partido.find_element(By.CSS_SELECTOR, "a")
                href = enlace.get_attribute("href")
                url2 = f"{href}/match-statistics/0"

                lineas = partido.text.split("\n")
                #print("Partido", len(lineas))
                #print(lineas)

                retirado = 'no'
                jugadorA = jugadorB = setsA = setsB = fecha = ""

                if len(lineas) >= 5:
                    jugadorA = lineas[1]
                    jugadorB = lineas[2]
                    setsA = lineas[3]
                    setsB = lineas[4]
                    fecha = lineas[0]
                    retirado = 'no'
                    print(len(lineas))

                    torneo_actual = {
                        "nombre":nombre,
                        "jugadorA": jugadorA,
                        "jugadorB": jugadorB,
                        "setsA": setsA,
                        "setsB": setsB,
                        "fecha": fecha,
                        "retirado": retirado
                    }

                else:
                    print("Hay menos de 5 lineas")
                    print(lineas)

                enlaces_partidos.append({
                    "url2": url2,
                    "info_partido": torneo_actual.copy()
                })
                    

            except Exception as e:
                print(f"Error extrayendo el enlace: {e}")
                continue

        for s, partido_data in enumerate(enlaces_partidos):
            url2 = partido_data['url2']
            torneo_info = partido_data['info_partido']
            print(f"Visitando partido {s}: {url2}")
            driver.get(url2)
            try:
                wait.until(EC.presence_of_element_located((By.CLASS_NAME, "section")))
                estadisticas = driver.find_elements(By.CLASS_NAME, "section")

                time.sleep(2)

                acesA = acesB = dobleA = dobleB = ""
                porcentajePrimerSaqueA = porcentajePrimerSaqueB = ""
                porcentajeJuegosGanadosPrimerSaqueA = juegosGanadosPrimerSaqueA = ""
                porcentajeJuegosGanadosPrimerSaqueB = juegosGanadosPrimerSaqueB = ""
                porcentajeJuegosGanadosSegundoSaqueA = JuegosGanadosSegundoSaqueA = ""
                porcentajeJuegosGanadosSegundoSaqueB = JuegosGanadosSegundoSaqueB = ""
                porcentajeBreakPointsSalvadosA = BreakPointsSalvadosA = ""
                porcentajeBreakPointsSalvadosB = BreakPointsSalvadosB = ""
                porcentajePuntosBreakConvertidosA = PuntosBreakConvertidosA = ""
                porcentajePuntosBreakConvertidosB = PuntosBreakConvertidosB = ""
                velocidadMediaPrimerSaqueA = velocidadMediaPrimerSaqueB = ""
                velocidadMediaSegundoSaqueA = velocidadMediaSegundoSaqueB = ""
                porcentajePuntosGanadosAlRestoConPrimerSaqueA = PuntosGanadosAlRestoConPrimerSaqueA = ""
                porcentajePuntosGanadosAlRestoConPrimerSaqueB = PuntosGanadosAlRestoConPrimerSaqueB = ""
                porcentajePuntosGanadosAlRestoConSegundoSaqueA = PuntosGanadosAlRestoConSegundoSaqueA = ""
                porcentajePuntosGanadosAlRestoConSegundoSaqueB = PuntosGanadosAlRestoConSegundoSaqueB = ""
                winnersA = winnersB = ""
                erroresNoForzadosA = erroresNoForzadosB = ""
                porcentajePuntosGanadosRedA = PuntosGanadosRedA = ""
                porcentajePuntosGanadosRedB = PuntosGanadosRedB = ""
                porcentajePuntosGanadosAlSaqueA = PuntosGanadosAlSaqueA = ""
                porcentajePuntosGanadosAlSaqueB = PuntosGanadosAlSaqueB = ""
                porcentajePuntosGanadosAlRestoA = PuntosGanadosAlRestoA = ""
                porcentajePuntosGanadosAlRestoB = PuntosGanadosAlRestoB = ""
                porcentajePuntosGanadosA = PuntosGanadosA = ""
                porcentajePuntosGanadosB = PuntosGanadosB = ""
                PuntosGanadosUltimas10BolasA = PuntosGanadosUltimas10BolasB = ""
                BolasDePartidoSalvadasA = BolasDePartidoSalvadasB = ""
                porcentajeJuegosGanadosAlSaqueA = JuegosGanadosAlSaqueA = ""
                porcentajeJuegosGanadosAlSaqueB = JuegosGanadosAlSaqueB = ""
                porcentajeJuegosGanadosAlRestoA = JuegosGanadosAlRestoA = ""
                porcentajeJuegosGanadosAlRestoB = JuegosGanadosAlRestoB = ""
                porcentajeJuegosGanadosA = JuegosGanadosA = ""
                porcentajeJuegosGanadosB = JuegosGanadosB = ""
                cuotaA = cuotaB = cuotaA_2 = cuotaB_2 = ""

                for x,est in enumerate(estadisticas):
                    print("Estadísticas:", x)
                    print(est.text)

                    
                    lineas = est.text.split("\n")
                    for i, line in enumerate(lineas):
                        l = line.lower()
                        if "aces" in l:
                            acesA = lineas[i - 1]
                            acesB = lineas[i + 1]
                        if "double faults" in l:
                            dobleA = lineas[i - 1]
                            dobleB = lineas[i + 1]
                        if "1st serve percentage" in l:
                            porcentajePrimerSaqueA = lineas[i - 1]
                            porcentajePrimerSaqueB = lineas[i + 1]
                        if "1st serve points won" in l:
                            porcentajeJuegosGanadosPrimerSaqueA = lineas[i - 2]
                            juegosGanadosPrimerSaqueA = lineas[i - 1]
                            porcentajeJuegosGanadosPrimerSaqueB = lineas[i + 1]
                            juegosGanadosPrimerSaqueB = lineas[i + 2]
                        if "2nd serve points won" in l:
                            porcentajeJuegosGanadosSegundoSaqueA = lineas[i - 2]
                            JuegosGanadosSegundoSaqueA = lineas[i - 1]
                            porcentajeJuegosGanadosSegundoSaqueB = lineas[i + 1]
                            JuegosGanadosSegundoSaqueB = lineas[i + 2]
                        if "break points saved" in l:
                            porcentajeBreakPointsSalvadosA = lineas[i - 2]
                            BreakPointsSalvadosA = lineas[i - 1]
                            porcentajeBreakPointsSalvadosB = lineas[i + 1]
                            BreakPointsSalvadosB = lineas[i + 2]
                        if "average 1st serve speed" in l:
                            velocidadMediaPrimerSaqueA = lineas[i - 1]
                            velocidadMediaPrimerSaqueB = lineas[i + 1]
                        if "average 2nd serve speed" in l:
                            velocidadMediaSegundoSaqueA = lineas[i - 1]
                            velocidadMediaSegundoSaqueB = lineas[i + 1]
                        if "1st return points won" in l:
                            porcentajePuntosGanadosAlRestoConPrimerSaqueA = lineas[i - 2]
                            PuntosGanadosAlRestoConPrimerSaqueA = lineas[i - 1]
                            porcentajePuntosGanadosAlRestoConPrimerSaqueB = lineas[i + 1]
                            PuntosGanadosAlRestoConPrimerSaqueB = lineas[i + 2]
                        if "2nd return points won" in l:
                            porcentajePuntosGanadosAlRestoConSegundoSaqueA = lineas[i - 2]
                            PuntosGanadosAlRestoConSegundoSaqueA = lineas[i - 1]
                            porcentajePuntosGanadosAlRestoConSegundoSaqueB = lineas[i + 1]
                            PuntosGanadosAlRestoConSegundoSaqueB = lineas[i + 2]
                        if "break points converted" in l:
                            porcentajePuntosBreakConvertidosA = lineas[i - 2]
                            PuntosBreakConvertidosA = lineas[i - 1]
                            porcentajePuntosBreakConvertidosB = lineas[i + 1]
                            PuntosBreakConvertidosB = lineas[i + 2]
                        if "winners" in l:
                            winnersA = lineas[i - 1]
                            winnersB = lineas[i + 1]
                        if "unforced errors" in l:
                            erroresNoForzadosA = lineas[i - 1]
                            erroresNoForzadosB = lineas[i + 1]
                        if "net points won" in l:
                            porcentajePuntosGanadosRedA = lineas[i - 2]
                            PuntosGanadosRedA = lineas[i - 1]
                            porcentajePuntosGanadosRedB = lineas[i + 1]
                            PuntosGanadosRedB = lineas[i + 2]
                        if "service points won" in l:
                            porcentajePuntosGanadosAlSaqueA = lineas[i - 2]
                            PuntosGanadosAlSaqueA = lineas[i - 1]
                            porcentajePuntosGanadosAlSaqueB = lineas[i + 1]
                            PuntosGanadosAlSaqueB = lineas[i + 2]
                        if "return points won" in l:
                            porcentajePuntosGanadosAlRestoA = lineas[i - 2]
                            PuntosGanadosAlRestoA = lineas[i - 1]
                            porcentajePuntosGanadosAlRestoB = lineas[i + 1]
                            PuntosGanadosAlRestoB = lineas[i + 2]
                        if "total points won" in l:
                            porcentajePuntosGanadosA = lineas[i - 2]
                            PuntosGanadosA = lineas[i - 1]
                            porcentajePuntosGanadosB = lineas[i + 1]
                            PuntosGanadosB = lineas[i + 2]
                        if "last 10 balls" in l:
                            PuntosGanadosUltimas10BolasA = lineas[i - 1]
                            PuntosGanadosUltimas10BolasB = lineas[i + 1]
                        if "match points saved" in l:
                            BolasDePartidoSalvadasA = lineas[i - 1]
                            BolasDePartidoSalvadasB = lineas[i + 1]
                        if "service games won" in l:
                            porcentajeJuegosGanadosAlSaqueA = lineas[i - 2]
                            JuegosGanadosAlSaqueA = lineas[i - 1]
                            porcentajeJuegosGanadosAlSaqueB = lineas[i + 1]
                            JuegosGanadosAlSaqueB = lineas[i + 2]
                        if "return games won" in l:
                            porcentajeJuegosGanadosAlRestoA = lineas[i - 2]
                            JuegosGanadosAlRestoA = lineas[i - 1]
                            porcentajeJuegosGanadosAlRestoB = lineas[i + 1]
                            JuegosGanadosAlRestoB = lineas[i + 2]
                        if "total games won" in l:
                            porcentajeJuegosGanadosA = lineas[i - 2]
                            JuegosGanadosA = lineas[i - 1]
                            porcentajeJuegosGanadosB = lineas[i + 1]
                            JuegosGanadosB = lineas[i + 2]
                        if "odds" in l:
                            if len(lineas) >= i + 7:
                                cuotaA = lineas[i + 6]
                                cuotaB = lineas[i + 7]
                        if "show more" in l:
                                cuotaA_2 = lineas[i - 2]
                                cuotaB_2 = lineas[i - 1]

                # Guardar en lista de diccionarios
                datos.append({

                    "nombre": torneo_info.get("nombre", ""),
                    "jugadorA": torneo_info.get("jugadorA", ""),
                    "jugadorB": torneo_info.get("jugadorB", ""),
                    "setsA": torneo_info.get("setsA", ""),
                    "setsB": torneo_info.get("setsB", ""),
                    "fecha": torneo_info.get("fecha", ""),
                    "retirado": torneo_info.get("retirado", ""),

                    "acesA":acesA,
                    "acesB":acesB,

                    "dobleFaltasA":dobleA,
                    "dobleFaltasB":dobleB,

                    "porcentajePrimerSaqueA":porcentajePrimerSaqueA,
                    "porcentajePrimerSaqueB":porcentajePrimerSaqueB,

                    "porcentajePuntosGanadosConPrimerSaqueA":porcentajeJuegosGanadosPrimerSaqueA,
                    "porcentajePuntosGanadosConPrimerSaqueB":porcentajeJuegosGanadosPrimerSaqueB,
                    "puntosGanadosConPrimerSaqueA":juegosGanadosPrimerSaqueA,
                    "puntosGanadosConPrimerSaqueB":juegosGanadosPrimerSaqueB,

                    "porcentajePuntosGanadosConSegundoSaqueA":porcentajeJuegosGanadosSegundoSaqueA,
                    "porcentajePuntosGanadosConSegundoSaqueB":porcentajeJuegosGanadosSegundoSaqueB,
                    "puntosGanadosConSegundoSaqueA":JuegosGanadosSegundoSaqueA,
                    "puntosGanadosConSegundoSaqueB":JuegosGanadosSegundoSaqueB,

                    "porcentajeBreakPointsSalvadosA":porcentajeBreakPointsSalvadosA,
                    "porcentajeBreakPointsSalvadosB":porcentajeBreakPointsSalvadosB,
                    "breakPointsSalvadosA":BreakPointsSalvadosA,
                    "breakPointsSalvadosB":BreakPointsSalvadosB,

                    "porcentajeBreakPointsA":porcentajePuntosBreakConvertidosA,
                    "porcentajeBreakPointsB":porcentajePuntosBreakConvertidosB,
                    "breakPointsA":PuntosBreakConvertidosA,
                    "breakPointsB":PuntosBreakConvertidosB,

                    "velocidadMediaPrimerSaqueA":velocidadMediaPrimerSaqueA,
                    "velocidadMediaPrimerSaqueB":velocidadMediaPrimerSaqueB,
                    "velocidadMediaSegundoSaqueA":velocidadMediaSegundoSaqueA,
                    "velocidadMediaSegundoSaqueB":velocidadMediaSegundoSaqueB,

                    "porcentajePuntosGanadosAlRestoConPrimerSaqueA": porcentajePuntosGanadosAlRestoConPrimerSaqueA,
                    "porcentajePuntosGanadosAlRestoConPrimerSaqueB": porcentajePuntosGanadosAlRestoConPrimerSaqueB,
                    "puntosGanadosAlRestoConPrimerSaqueA": PuntosGanadosAlRestoConPrimerSaqueA,
                    "puntosGanadosAlRestoConPrimerSaqueB": PuntosGanadosAlRestoConPrimerSaqueB,

                    "porcentajePuntosGanadosAlRestoConSegundoSaqueA": porcentajePuntosGanadosAlRestoConSegundoSaqueA,
                    "porcentajePuntosGanadosAlRestoConSegundoSaqueB": porcentajePuntosGanadosAlRestoConSegundoSaqueB,
                    "puntosGanadosAlRestoConSegundoSaqueA": PuntosGanadosAlRestoConSegundoSaqueA,
                    "puntosGanadosAlRestoConSegundoSaqueB": PuntosGanadosAlRestoConSegundoSaqueB,

                    "winnersA":winnersA,
                    "winnersB":winnersB,

                    "erroresNoForzadosA":erroresNoForzadosA,
                    "erroresNoForzadosB":erroresNoForzadosB,

                    "porcentajePuntosGanadosRedA":porcentajePuntosGanadosRedA,
                    "porcentajePuntosGanadosRedB":porcentajePuntosGanadosRedB,
                    "puntosGanadosRedA":PuntosGanadosRedA,
                    "puntosGanadosRedB":PuntosGanadosRedB,

                    "porcentajePuntosGanadosSaqueA":porcentajePuntosGanadosAlSaqueA,
                    "porcentajePuntosGanadosSaqueB":porcentajePuntosGanadosAlSaqueB,
                    "puntosGanadosSaqueA":PuntosGanadosAlSaqueA,
                    "puntosGanadosSaqueB":PuntosGanadosAlSaqueB,

                    "porcentajePuntosGanadosRestoA":porcentajePuntosGanadosAlRestoA,
                    "porcentajePuntosGanadosRestoB":porcentajePuntosGanadosAlRestoB,
                    "puntosGanadosRestoA":PuntosGanadosAlRestoA,
                    "puntosGanadosRestoB":PuntosGanadosAlRestoB,

                    "porcentajePuntosGanadosA":porcentajePuntosGanadosA,
                    "porcentajePuntosGanadosB":porcentajePuntosGanadosB,
                    "puntosGanadosA":PuntosGanadosA,
                    "puntosGanadosB":PuntosGanadosB,

                    "puntosGanadosUltimas10BolasA":PuntosGanadosUltimas10BolasA,
                    "puntosGanadosUltimas10BolasB":PuntosGanadosUltimas10BolasB,

                    "puntosDePartidoSalvadosA":BolasDePartidoSalvadasA,
                    "puntosDePartidoSalvadosB":BolasDePartidoSalvadasB,

                    "porcentajeJuegosGanadosSaqueA":porcentajeJuegosGanadosAlSaqueA,
                    "porcentajeJuegosGanadosSaqueB":porcentajeJuegosGanadosAlSaqueB,
                    "juegosGanadosSaqueA":JuegosGanadosAlSaqueA,
                    "juegosGanadosSaqueB":JuegosGanadosAlSaqueB,

                    "porcentajeJuegosGanadosRestoA":porcentajeJuegosGanadosAlRestoA,
                    "porcentajeJuegosGanadosRestoB":porcentajeJuegosGanadosAlRestoB,
                    "juegosGanadosRestoA":JuegosGanadosAlRestoA,
                    "juegosGanadosRestoB":JuegosGanadosAlRestoB,

                    "porcentajeJuegosGanadosA":porcentajeJuegosGanadosA,
                    "porcentajeJuegosGanadosB":porcentajeJuegosGanadosB,
                    "juegosGanadosA":JuegosGanadosA,
                    "juegosGanadosB":JuegosGanadosB,

                    "cuotaA":cuotaA,
                    "cuotaB":cuotaB,

                    "cuotaA_2":cuotaA_2,
                    "cuotaB_2":cuotaB_2

                })


            except Exception as e:
                print(f"Error obteniendo estadísticas del partido {s}: {e}")


except Exception as e:
    print("Error:", e)
    
finally:
    driver.quit()
    # Exportar a CSV
    df = pd.DataFrame(datos)
    display(df)
    #df.to_csv("partidos_tenis2.csv", index=False)
    #print("Datos guardados de", fecha_actual)

Dentro de https://www.flashscore.com/tennis/itf-women-singles/w50-guimaraes/results/
Número de partidos: 6 en el enlace 0 de 51 enlaces
9
9
9
13
9
9
Visitando partido 0: https://www.flashscore.com/match/tennis/EsHUJMDN/#/match-summary/match-statistics/0
Estadísticas: 0
SCORE
Glushko L.
(4)
0
5
3
Vlasselaer C.
2
7
6
MATCH TIME
1:46
1:03
0:43
Estadísticas: 1
ODDS
HOME/AWAY
OVER/UNDER
ASIAN HANDICAP
1
2
1.36
3.00
Show more
Dobla tu depósito hasta 200€
Estadísticas: 2
1
2
1.36
3.00
Show more
Dobla tu depósito hasta 200€
Visitando partido 1: https://www.flashscore.com/match/tennis/0vie5NbU/#/match-summary/match-statistics/0
Estadísticas: 0
Error obteniendo estadísticas del partido 1: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=137.0.7151.69); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x0x9937

,nombre,jugadorA,jugadorB,setsA,setsB,fecha,retirado,acesA,acesB,dobleFaltasA,...,juegosGanadosRestoA,juegosGanadosRestoB,porcentajeJuegosGanadosA,porcentajeJuegosGanadosB,juegosGanadosA,juegosGanadosB,cuotaA,cuotaB,cuotaA_2,cuotaB_2
0,ITF W50 Guimaraes Women,Glushko L.,Vlasselaer C.,0,2,10.06. 18:00,no,,,,...,,,,,,,1.36,3.00,1.36,3.00
1,ITF W50 Guimaraes Women,Inoue H.,Matias M.,2,0,10.06. 16:30,no,,,,...,,,,,,,,,,
2,ITF W50 Guimaraes Women,Ewald W.,Raina A.,2,1,10.06. 15:15,no,,,,...,,,,,,,1.50,2.50,1.50,2.50
3,ITF W75 Ceska Lipa Women,Zantedeschi A.,Curmi F.,2,0,10.06. 13:25,no,,,,...,,,,,,,2.20,1.62,2.20,1.62
4,ITF W75 Ceska Lipa Women,Zelnickova R.,Fossa Huergo N.,2,1,10.06. 13:20,no,,,,...,,,,,,,1.73,Show more,2.00,1.73
5,ITF W75 Ceska Lipa Women,Kan V.,Gimbrere J.,0,2,10.06. 11:55,no,,,,...,,,,,,,1.73,2.00,1.73,2.00
6,ITF W75 Ceska Lipa Women,Karatancheva L.,Jokic K.,2,0,10.06. 10:10,no,,,,...,,,,,,,1.57,2.25,1.57,2.25
7,ITF W75 Ceska Lipa Women,Lee G.,Yao X.,2,1,10.06. 10:10,no,,,,...,,,,,,,1.44,2.63,1.44,2.63
8,ITF W75 Ceska Lipa Women,Makarova E.,Struplova J.,2,0,10.06. 10:10,no,,,,...,,,,,,,Show more,Dobla tu depósito hasta 200€,2.00,1.73


In [66]:
for i in df.columns:
    print(df[i].value_counts())

nombre
ITF W75 Ceska Lipa Women    7
ITF W50 Guimaraes Women     5
Name: count, dtype: int64
jugadorA
10.06. 10:10    3
10.06. 16:30    2
10.06. 13:25    2
10.06. 18:00    1
10.06. 14:55    1
10.06. 15:15    1
10.06. 13:20    1
10.06. 11:55    1
Name: count, dtype: int64
jugadorB
Glushko L.         1
Garcia M.          1
Inoue H.           1
Ewald W.           1
Abe H.             1
Fajmonova S. M.    1
Zantedeschi A.     1
Zelnickova R.      1
Kan V.             1
Karatancheva L.    1
Lee G.             1
Makarova E.        1
Name: count, dtype: int64
setsA
2    9
0    3
Name: count, dtype: int64
setsB
0    6
2    3
1    3
Name: count, dtype: int64
fecha
6    7
4    2
5    1
7    1
1    1
Name: count, dtype: int64
retirado
si    12
Name: count, dtype: int64
acesA
    12
Name: count, dtype: int64
acesB
    12
Name: count, dtype: int64
dobleFaltasA
    12
Name: count, dtype: int64
dobleFaltasB
    12
Name: count, dtype: int64
porcentajePrimerSaqueA
    12
Name: count, dtype: int64
porce